In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
%%capture


!pip install datasets
!pip install evaluate

In [45]:
from google.colab import userdata
import os

sec_key = userdata.get('HF_TOKEN')
os.environ['HUGGINGFACEHUB_API_TOKEN'] = sec_key

In [46]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files="/content/drive/MyDrive/Scripts/30 Days Of Bert/Day06/train.csv")
dataset["train"][1]

{'text': 'veio umas teorias muito loucas na minha cabeça agora, pqp to assustado',
 'label': 1}

In [47]:

def conver2pandas(data):
    data.set_format(type="pandas")
    df = data["train"][:]

    return df


dataframe = conver2pandas(dataset)
dataframe

,text,label
0,"rt @user olha quem chegouuuuu, nossos queridin...",0
1,veio umas teorias muito loucas na minha cabeça...,1
2,@user @user 😂😂😂😂mais nao tinha falado ontem qu...,0
3,rt @user quer ser filha da puta logo comigo qu...,1
4,vai besta 😂😂😂😂 casquei com a ultima foto,1
...,...,...
16795,performer da nação caralho,1
16796,"vôlei feminino é foda né, pqp, só vem tóquio",1
16797,"@user cara de pau, quem desrespeita a constitu...",1
16798,duas das grandes atletas do frescobol mundial....,0


In [48]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [49]:
Train, Test = train_test_split(dataframe, test_size=0.3, random_state=0)
Train.shape, Test.shape

((11760, 2), (5040, 2))

In [50]:
n_classes = len(Train["label"].value_counts())
n_classes

2

In [51]:
from datasets import Dataset
Train = Dataset.from_pandas(Train)
Test = Dataset.from_pandas(Test)

In [52]:
Train

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 11760
})

In [53]:
Test

Dataset({
    features: ['text', 'label', '__index_level_0__'],
    num_rows: 5040
})

In [54]:
from transformers import AutoTokenizer


tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_train = Train.map(tokenize_function, batched=True)
tokenized_test = Test.map(tokenize_function, batched=True)


Map:   0%|          | 0/11760 [00:00<?, ? examples/s]

Map:   0%|          | 0/5040 [00:00<?, ? examples/s]

In [55]:
small_train_dataset = tokenized_train.shuffle(seed=42)
small_eval_dataset = tokenized_test.shuffle(seed=42)

In [56]:
from transformers import AutoModelForSequenceClassification


model = AutoModelForSequenceClassification.from_pretrained(
    "google-bert/bert-base-cased",
    num_labels=n_classes,
    )


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [57]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [58]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [59]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [62]:
training_args = TrainingArguments(
    output_dir="test_trainer",
    eval_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    logging_steps=1,
    num_train_epochs=10,
)

In [63]:
from transformers import TrainingArguments, Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics

)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
